<a href="https://colab.research.google.com/github/MeowSky49887/Napha/blob/main/APIs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Napha APIs**

---

This is Combined APIs for *Napha*

Including
*   LLaMa CPP
*   VOICEVOX Engine

For *Napha* Application please download at https://github.com/MeowSky49887/Napha

---

## Setting up Napha APIs

In [ ]:
# @title ###Step1: Setting Up Python 3.11 Environment in Google Colab

# Check current Python 3 version
!python3 --version

# Configure Python 3 alternatives
!sudo update-alternatives --config python3

# Update package list
!sudo apt-get update -y

# Install Python 3.11 and development tools
!sudo apt-get install -y python3.11 python3.11-dev python3.11-distutils libpython3.11-dev

# Set Python 3.11 as default
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.11 2

# Verify Python 3 version
!python3 --version

# Ensure distutils for Python 3.11 is installed
!sudo apt-get install -y python3.11-distutils

# Download get-pip.py script
!wget https://bootstrap.pypa.io/get-pip.py

# Install pip using the downloaded script
!python3.11 get-pip.py

# Install necessary Python packages
!python3.11 -m pip install ipython ipython_genutils ipykernel jupyter_console prompt_toolkit httplib2 astor

In [ ]:
# @title ###Step:2 Setting Up VOICEVOX Engine and LLaMA CPP

# Download the voicevox engine archive
!wget 'https://github.com/VOICEVOX/voicevox_engine/releases/download/0.19.1/voicevox_engine-linux-cpu-0.19.1.7z.001'

# Install p7zip to extract the .7z archive
!apt-get update -y
!apt-get install -y p7zip-full

# Extract the archive
!7z x 'voicevox_engine-linux-cpu-0.19.1.7z.001'

# Clean up the downloaded archive
!rm 'voicevox_engine-linux-cpu-0.19.1.7z.001'

# Clone voicevox engine repository
!git clone https://github.com/VOICEVOX/voicevox_engine -q

# Install necessary packages for voicevox engine and llama.cpp public url
!sed "s/;.*//" voicevox_engine/requirements.txt | pip install -r /dev/stdin
!pip install llama-cpp-python fastapi huggingface_hub
!npm install -g localtunnel -q

In [ ]:
# @title ###Step3: Creating a FastAPI Chat Application with specific LLaMA CPP Model

# Specify the content of the Python file
file_content = """
from llama_cpp import Llama
from fastapi import FastAPI
from pydantic import BaseModel
from typing import List
from fastapi.responses import JSONResponse

class MessageItem(BaseModel):
    role: str
    content: str

class MessageList(BaseModel):
    messages: List[MessageItem]

app = FastAPI()

# Load the pre-trained model
guanaco = Llama.from_pretrained(
    repo_id="RepositoryID",
    filename="FileName",
    verbose=False
)

@app.get("/")
def root():
    return {"Hello": "World"}

@app.post("/chat")
async def chat(history: MessageList):
    # Convert the input messages to the format expected by the model
    messages = [{"role": msg.role, "content": msg.content} for msg in history.messages]

    # Generate a response using the model
    response = guanaco.create_chat_completion(messages=messages)

    # Return the model's response
    return JSONResponse(content=response)
"""

RepositoryID = "RichardErkhov/TheTravellingEngineer_-_llama2-7b-chat-hf-guanaco-gguf" # @param {type:"string"}
FileName = "llama2-7b-chat-hf-guanaco.Q4_K_M.gguf" # @param {type:"string"}

file_content = file_content.replace("RepositoryID", RepositoryID).replace("FileName", FileName)

# Write the content to a Python file
with open('main.py', 'w') as file:
    file.write(file_content)

print("Python file 'main.py' created successfully.")

## Running Napha APIs

In [ ]:
# @title ###Just Click Run ✔

import threading
import subprocess

def run_fastapi():
    !fastapi run main.py

def run_voicevox():
    !python voicevox_engine/run.py --voicevox_dir='linux-cpu' --host 0.0.0.0 --port 50021

def run_localtunnel_port8000():
    !lt --port 8000 --subdomain "napha-llama"

def run_localtunnel_port50021():
    !lt --port 50021 --subdomain "napha-voicevox"\

# Create threads for each command
thread_fastapi = threading.Thread(target=run_fastapi)
thread_voicevox = threading.Thread(target=run_voicevox)
thread_localtunnel_8000 = threading.Thread(target=run_localtunnel_port8000)
thread_localtunnel_50021 = threading.Thread(target=run_localtunnel_port50021)

# Start all threads
thread_fastapi.start()
thread_voicevox.start()
thread_localtunnel_8000.start()
thread_localtunnel_50021.start()

# Join all threads to wait for completion
thread_fastapi.join()
thread_voicevox.join()
thread_localtunnel_8000.join()
thread_localtunnel_50021.join()

print("All threads finished execution")
